In [ ]:
# dataset preprocess
import os
import csv
import numpy as np 

train_data_dir = './train_data'
train_img_dir = train_data_dir + '/img'
train_meta_dir = train_data_dir + '/metadata'
train_ctrl_dir = train_data_dir + '/ctrl'

scenes = [f.train_ctrl_dir for f in os.scandir(train_ctrl_dir) if f.is_dir()]
scenes_num = len(scenes)

eval_list = [3]  

def minmax(vec, maxval, minval):
    return (vec-minval)/(maxval - minval)

ctrl_vec_list = []
# create a csv to document the file
with open('train.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['ctrl', 'img_exp', 'img_cur', 'meta_exp', 'meta_cur'])    
    writer.writeheader()

    for scene_idx in range(scenes_num):
        if scene_idx in eval_list:
            continue
        filename = scenes[scene_idx][len(train_ctrl_dir)+1:]
        ctrl_subdir = scenes[scene_idx]
        img_subdir = os.path.join(train_img_dir, filename)
        meta_subdir = os.path.join(train_meta_dir, filename)
        sub_ctrl_dir = os.listdir(ctrl_subdir)
        # sub_img_dir = os.listdir(img_subdir)
        # sub_meta_dir = os.listdir(meta_subdir)
        for i in range(len(sub_ctrl_dir)):
            # calculate the vector of ctrl
            ctrl = np.loadtxt(os.path.join(ctrl_subdir, sub_ctrl_dir[i]))
            ctrl_vec_list.append(np.sum(ctrl, 0)[-4:]) # how to normalize the label to 0-1?? use minmax
            writer.writerow(
            {'ctrl' : os.path.join(ctrl_subdir, str(i)+'vec.txt'), 'img_exp': os.path.join(img_subdir, str(i)+'after.jpg'), \
            'img_cur': os.path.join(img_subdir, str(i)+'err.jpg'), 'meta_exp': os.path.join(meta_subdir, str(i)+'after.txt'), \
            'meta_cur': os.path.join(meta_subdir, str(i)+'err.txt')})


with open('val.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['ctrl', 'img_exp', 'img_cur', 'meta_exp', 'meta_cur'])    
    writer.writeheader()
    for scene_idx in range(scenes_num):
        if scene_idx in eval_list:
            filename = scenes[scene_idx][len(train_ctrl_dir)+1:]
            ctrl_subdir = scenes[scene_idx]
            img_subdir = os.path.join(train_img_dir, filename)
            meta_subdir = os.path.join(train_meta_dir, filename)
            sub_ctrl_dir = os.listdir(ctrl_subdir)
            # sub_img_dir = os.listdir(img_subdir)
            # sub_meta_dir = os.listdir(meta_subdir)
            for i in range(len(sub_ctrl_dir)):
                ctrl = np.loadtxt(os.path.join(ctrl_subdir, sub_ctrl_dir[i]))
                ctrl_vec_list.append(np.sum(ctrl, 0)[-4:]) # how to normalize the label to 0-1?? use minmax
                writer.writerow(
                {'ctrl' : os.path.join(ctrl_subdir, str(i)+'vec.txt'), 'img_exp': os.path.join(img_subdir, str(i)+'after.jpg'), \
                'img_cur': os.path.join(img_subdir, str(i)+'err.jpg'), 'meta_exp': os.path.join(meta_subdir, str(i)+'after.txt'), \
                'meta_cur': os.path.join(meta_subdir, str(i)+'err.txt')})


ctrl_vec_list = np.array(ctrl_vec_list)
maxval = np.max(ctrl_vec_list, 0)
minval = np.min(ctrl_vec_list, 0)
for scene_idx in range(scenes_num):
    ctrl_subdir = scenes[scene_idx]
    sub_ctrl_dir = os.listdir(ctrl_subdir)
    for i in range(len(sub_ctrl_dir)):
        ctrl = np.loadtxt(os.path.join(ctrl_subdir, sub_ctrl_dir[i]))
        np.savetxt(os.path.join(ctrl_subdir, str(i)+'vec.txt'), minmax(np.sum(ctrl, 0)[-4:], maxval, minval))





In [24]:
import os
import os.path
import sys
import torch
import torch.utils.data as data
import numpy as np
import pandas as pd
from PIL import Image
import torchvision as tv

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class imgcontroldataset(data.Dataset):
    def __init__(self, csv_dir, mode = 'train', image_size = (224, 224)):
        super(imgcontroldataset, self).__init__()
        self.datacsv = pd.read_csv(os.path.join(csv_dir, "%s.csv" % mode))
        self.image_size = image_size
        
    def __len__(self):
        return 3*len(self.datacsv)  # augment the dataset for 3 times, randomness from randomcrop
        
    def __getitem__(self, idx):
        idx = idx % len(self.datacsv)
        
        img_cur_path = self.datacsv.iloc[idx]['img_cur']
        img_cur = Image.open(img_cur_path).convert('RGB')
        img_exp_path = self.datacsv.iloc[idx]['img_exp']
        img_exp = Image.open(img_exp_path).convert('RGB')  
        meta_cur_path = self.datacsv.iloc[idx]['meta_cur']
        meta_cur = np.loadtxt(meta_cur_path)
        meta_exp_path = self.datacsv.iloc[idx]['meta_exp']
        meta_exp = np.loadtxt(meta_exp_path)
        ctrl_path = self.datacsv.iloc[idx]['ctrl']
        ctrl = np.loadtxt(ctrl_path)
        
        # img transformation
        transform = tv.transforms.Compose([        
            tv.transforms.Resize(min(image_size)),    # fix the ratio and keep the smaller edge to 224 according to Relative **
            tv.transforms.RandomCrop(image_size),
            tv.transforms.ColorJitter(),  # randomly change hue, contrast illumination
            tv.transforms.ToTensor(),    
            tv.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),    # copied from pytorch tutorial imagenet
            ])
        img1 = transform(img_exp)
        img2 = transform(img_cur)
        img = torch.cat((img1, img2), dim = 0)  # The dimension 0 is the RGB channel
        meta = torch.cat((torch.from_numpy(meta_exp)[:, None], torch.from_numpy(meta_cur)[:, None]), dim = 1)
        ctrl = torch.from_numpy(ctrl)
        
        return img, meta, ctrl
        